In [73]:
import numpy as np
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gc
import time
import pytz
import datetime
import logging
from tqdm import tqdm
import traceback
import docx2txt


In [47]:
df = pd.read_excel('GMAT_All_QS.xlsx')

In [48]:
df_sub = pd.read_excel('Gmat_practiece_verbal.xlsx')

In [49]:
new_column_names = {col: col.lower().replace(' ', '_') for col in df.columns}
df = df.rename(columns = new_column_names)
df.columns 

Index(['question_id', 'main_question_stem', 'subquestion1', 'subquestion2',
       'subquestion3', 'answer_1', 'answer_2', 'answer_3', 'answer_4',
       'answer_5', 'correct_answer', 'category', 'level'],
      dtype='object')

In [50]:
df_sub.columns

Index(['passage_content', 'Question Number', 'Question Content', 'choice_(A)',
       'choice_(B)', 'choice_(C)', 'choice_(D)', 'choice_(E)', 'ans'],
      dtype='object')

In [51]:
new_column_names = {'passage_content':'main_question_stem', 'Question Number':'question_id', 'Question Content':'subquestion1', 'choice_(A)':'answer_1',
       'choice_(B)':"answer_2", 'choice_(C)':'answer_3', 'choice_(D)':'answer_4', 'choice_(E)':'answer_5', 'ans':'correct_answer'}

# new_column_names = {'Question Content':'main_question_stem', 'Question Number':'question_id', 'choice_(A)':'answer_1',
#        'choice_(B)':"answer_2", 'choice_(C)':'answer_3', 'choice_(D)':'answer_4', 'choice_(E)':'answer_5', 'ans':'correct_answer'}

# new_column_names = {'Question Number':'question_id', 'Question Content':'main_question_stem', 'choice_(A)':'answer_1',
#        'choice_(B)':"answer_2", 'choice_(C)':'answer_3', 'choice_(D)':'answer_4', 'choice_(E)':'answer_5', 'ans':'correct_answer'}

# new_column_names = {'Question Number':'question_id', 'Question Content':'main_question_stem', 'choice_A.':'answer_1',
#        'choice_B.':"answer_2", 'choice_C.':'answer_3', 'choice_D.':'answer_4', 'choice_E.':'answer_5', 'ans':'correct_answer'}
df_sub = df_sub.rename(columns = new_column_names)

In [52]:
new_column_names = {col: col.lower().replace(' ', '_') for col in df_sub.columns}
df_sub = df_sub.rename(columns = new_column_names)
df_sub.columns

Index(['main_question_stem', 'question_id', 'subquestion1', 'answer_1',
       'answer_2', 'answer_3', 'answer_4', 'answer_5', 'correct_answer'],
      dtype='object')

In [53]:
df_sub

,main_question_stem,question_id,subquestion1,answer_1,answer_2,answer_3,answer_4,answer_5,correct_answer
0,"Questions 1–6 refer to the following passage, ...",1,The main idea of the first paragraph is bes...,The factors that make an area desirable are...,The Hawaiian Islands would not exist if not...,"Floods, volcanoes, and earthquakes pose thr...",Humans must learn to live with natural haza...,Natural hazards are most prevalent in areas...,A
1,"Questions 1–6 refer to the following passage, ...",2,Which of the following might the author of ...,Desecration of a library due to vandalism,Damage to a school building in a fire,Loss of retail clothing sales due to a mall...,Death of a ranch’s livestock due to vol- ca...,Destruction of a pavilion due to a hurricane,C
2,"Questions 1–6 refer to the following passage, ...",3,"According to the passage, an important dis...","risks occur naturally, while hazards arise ...","hazards result from risks, and risks result...","hazards can lead to disasters, while risks ...","hazards are not under human control, while ...",risks are harder to quantify than hazards,D
3,"Questions 1–6 refer to the following passage, ...",4,The passage is primarily concerned with,describing the causes and impacts of natura...,assessing the impact that disasters render...,depicting the various ways human beings may...,raising awareness of the loss of human live...,explaining that natural disasters are not u...,A
4,"Questions 1–6 refer to the following passage, ...",5,Which of the following best describes the p...,It uses numerical data and metrics to descr...,It emphasizes how little is actually known ...,It outlines the differences between hazards...,It provides sensory details about spe- cifi...,It applies statistical data to emphasize th...,E
5,"Questions 1–6 refer to the following passage, ...",6,Which of the following logically follows th...,The number of unreported deaths in the 2010...,"In the years 2010 and 2004, there were more...",The number of deaths due to natural disaste...,Economic costs are more frequently unreport...,"Areas of high hazard, such as Japan’s Tohok...",E
6,NaN,7,Before the primary school can open to the p...,"must finish construction, cleaning, and saf...","must finish construction of, cleaning, and ...","must finish constructing, cleaning and safe...","must finish constructing, cleaning, and saf...","must finish construction, cleaning and safe...",D
7,NaN,8,So many accounts of this continents’ past...,continents’ past begin with Europeans strid...,continent’s past begin with Europeans strid...,continents’ past begin with Europeans strid...,continent’s past begins with Europeans stri...,continent’s past begins with Europeans stri...,B
8,NaN,9,"Injera, an East African flatbread, has been...","are still used by many Ethiopians today, wh...","are still used by many Ethiopians today, wh...","it is still used today by many Ethiopians, ...",is still used today by many Ethiopians to f...,"is still used by many Ethiopians today, who...",D
9,NaN,10,The size of oceanic waves is a function of ...,The degree of erosion to which a coast- lin...,The size of oceanic waves will not fluc - t...,The fetch of winds is related to the shape ...,The size of oceanic waves is related to the...,The average velocity of the wind in an area...,A


In [54]:
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   main_question_stem  15 non-null     object
 1   question_id         41 non-null     int64 
 2   subquestion1        41 non-null     object
 3   answer_1            41 non-null     object
 4   answer_2            41 non-null     object
 5   answer_3            41 non-null     object
 6   answer_4            41 non-null     object
 7   answer_5            41 non-null     object
 8   correct_answer      41 non-null     object
dtypes: int64(1), object(8)
memory usage: 3.0+ KB


In [55]:
df = df.reset_index(drop=True)
df_sub = df_sub.reset_index(drop=True)

df = pd.concat([df, df_sub], ignore_index=True)

In [56]:
df

,question_id,main_question_stem,subquestion1,subquestion2,subquestion3,answer_1,answer_2,answer_3,answer_4,answer_5,correct_answer,category,level
0,1,"In a locality, there are thrice as many buses ...",NaN,NaN,NaN,0.225,0.25,0.3,0.333,0.36,A,Percentage,1.0
1,2,"In a school, 20% of the boys learn French whil...",NaN,NaN,NaN,20,24,30,36,45,C,Percentage,1.0
2,1,A and B need to do the laundry since their was...,NaN,NaN,NaN,1/12,1/9,1/8,1/6,1/3,B,Rate and Work,1.0
3,2,"A can write 10 programming codes in 15 hours, ...",NaN,NaN,NaN,1/5,3/10,1/3,9/20,1/2,D,Rate and Work,1.0
4,3,A and B can paint 3 walls in 2 hours. Along wi...,NaN,NaN,NaN,1.5 hours,1.8 hours,2 hours,2.5 hours,3 hours,A,Rate and Work,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,37,NaN,"A new kind of primate, the fat-tailed lemur...",NaN,NaN,The statement challenges the long-held beli...,The statement accuses scientists who have s...,The statement highlights the importance of ...,The statement calls into question the assum...,The statement sets up a rival theory so tha...,C,NaN,NaN
532,38,NaN,"When we think about peoples of the past, we...",NaN,NaN,we intuitively try to imagine what he or sh...,we intuitively try to imagine what they wou...,we intuitively try to imagine what he or sh...,we intuitively try to imagine what they wou...,we intuitively try to imagine what they wou...,B,NaN,NaN
533,39,NaN,Motorcyclists are told to always wear helme...,NaN,NaN,"My European client calls once a week, alway...",This sunscreen allows me to stay in the sun...,"The odds are 1,000 to 1 against winning the...",Seat belts protect passengers in auto - mob...,Top business schools accept one out of ever...,A,NaN,NaN
534,40,NaN,Even though Carter had fewer quarters in hi...,NaN,NaN,Even though Carter had fewer quarters in hi...,Even though Carter had fewer quarters than ...,Even though Carter had less quarters than h...,Even though Carter had less quarters in his...,Even though Carter had fewer quarters than ...,A,NaN,NaN


In [57]:
df.to_excel('GMAT_All_QS.xlsx',index = False)

In [59]:
df = pd.read_excel('GMAT_All_QS.xlsx')

In [68]:
collis = df.iloc[:,1:-1].columns.tolist()

In [87]:
for col in collis:
    df[col] = df[col].str.strip(' ')
    df[col] = df[col].str.replace(r'\s+', ' ')

C:\Users\PM\AppData\Local\Temp\ipykernel_10196\4277067208.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r'\s+', ' ')


In [88]:
df.to_excel('GMAT_All_QS.xlsx',index = False)